In [ ]:
import redis
from redis.commands.search.field import VectorField, TagField
from redis.commands.search.query import Query
import os

In [ ]:
pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
r = redis.Redis(connection_pool=pool, ssl=False)

In [ ]:
r.ping()

In [ ]:
r.keys('*')

In [ ]:
r.flushdb()

In [ ]:
r.keys('*')

In [ ]:
from deepface import DeepFace
 
embeddings = []
for dirpath, dirnames, filenames in os.walk("db/"):
    for filename in filenames:
        if ".jpg" in filename:
            img_path = f"{dirpath}{filename}"
            embedding = DeepFace.represent(
                img_path=img_path,
                model_name="Facenet",
                detector_backend="mtcnn",
            )[0]["embedding"]
            embeddings.append((img_path, embedding))

In [ ]:
pipeline = r.pipeline(transaction=False)
for img_path, embedding in tqdm(embeddings):
    key = img_path.split("/")[-1]
    value = np.array(embedding).astype(np.float32).tobytes()
    pipeline.hset(key, mapping = {"embedding": value})
# -------------------------------------
pipeline_results = pipeline.execute()

In [ ]:
min_value = -10
max_value = 10
size = 1000000
dims = 128
 
synthetic_vectors = np.random.uniform(min_value, max_value, size=(size, dims))
 
pipeline = r.pipeline(transaction=False)
for idx, synthetic_vector in enumerate(synthetic_vectors):
   key = f"synthetic_{idx}.jpg"
   value = np.array(synthetic_vector).astype(np.float32).tobytes()
   pipeline.hset(key, mapping = {"embedding": value})
# -------------------------------------
pipeline_results = pipeline.execute()

In [ ]:
r.ft().create_index(
    [
        VectorField(
            "embedding",
            "HNSW",
            {
                "TYPE": "FLOAT32",
                "DIM": 128,
                "DISTANCE_METRIC": "L2",
            },
        )
    ]
)

In [ ]:
target_embedding = DeepFace.represent(
    img_path="target.jpg",
    model_name="Facenet",
    detector_backend="mtcnn"
)[0]["embedding"]
 
query_vector = np.array(target_embedding).astype(np.float32).tobytes()

In [ ]:
k = 3
 
base_query = f"*=&amp;amp;gt;[KNN {k} @embedding $query_vector AS distance]"
query = Query(base_query).return_fields("distance").sort_by("distance").dialect(2)
results = r.ft().search(query, query_params={"query_vector": query_vector})
 
for idx, result in enumerate(results.docs):
    print(
        f"{idx + 1}th nearest neighbor is {result.id} with distance {result.distance}"
    )